In [2]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/81/4d/bcbee0676dc06f0b014a030cb928e158c96daafb872a01ddee7e60a8c998/folium-0.19.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/f8/9d/91cddd38bd00170aad1a4b198c47b4ed716be45c234e09b835af41f4e717/branca-0.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for xyzservices from https://files.pythonhosted.org/packages/9a/6e/49408735dae940a0c1c225c6b908fd83bd6e3f5fae120f865754e72f78cb/xyzservices-2025.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 110.9/110.9 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 88.4/88.4 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install fuzzywuzzy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for fuzzywuzzy from https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 125.4/125.4 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

In [27]:
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter



In [190]:
transactions = pd.read_csv('C:/Users/DSHA-EXT/Desktop/Dubai Land Data/Transactions (1).csv')

In [9]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427477 entries, 0 to 1427476
Data columns (total 46 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   transaction_id        1427477 non-null  object 
 1   procedure_id          1427477 non-null  int64  
 2   trans_group_id        1427477 non-null  int64  
 3   trans_group_ar        1427477 non-null  object 
 4   trans_group_en        1427477 non-null  object 
 5   procedure_name_ar     1427477 non-null  object 
 6   procedure_name_en     1427477 non-null  object 
 7   instance_date         1427477 non-null  object 
 8   property_type_id      1427477 non-null  int64  
 9   property_type_ar      1427477 non-null  object 
 10  property_type_en      1427477 non-null  object 
 11  property_sub_type_id  1124948 non-null  float64
 12  property_sub_type_ar  1124948 non-null  object 
 13  property_sub_type_en  1124948 non-null  object 
 14  property_usage_ar     1427477 non-

In [192]:
transactions['area_name_en'].nunique()

256

In [191]:
len(transactions)

1427477

In [48]:
# Initialize geocoder
geolocator = Nominatim(user_agent="dubai_map")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [193]:
area_names = pd.read_excel('C:/Users/DSHA-EXT/Desktop/Dubai Land Data/Python_Output_2_area_Lat_Long.xlsx')

In [196]:
area_names['Area Name'].nunique()

142

In [177]:
df = pd.merge(transactions, area_names, how='left', left_on='area_name_en', right_on='Area Name')
len(df)


1427477

In [197]:
exceptions = df[df['Latitude'].isnull() | df['Longitude'].isnull()]

In [198]:
len(exceptions)

21602

In [208]:
exceptions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21602 entries, 1 to 1427464
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           21602 non-null  object 
 1   procedure_id             21602 non-null  int64  
 2   trans_group_id           21602 non-null  int64  
 3   trans_group_ar           21602 non-null  object 
 4   trans_group_en           21602 non-null  object 
 5   procedure_name_ar        21602 non-null  object 
 6   procedure_name_en        21602 non-null  object 
 7   instance_date            21602 non-null  object 
 8   property_type_id         21602 non-null  int64  
 9   property_type_ar         21602 non-null  object 
 10  property_type_en         21602 non-null  object 
 11  property_sub_type_id     4172 non-null   float64
 12  property_sub_type_ar     4172 non-null   object 
 13  property_sub_type_en     4172 non-null   object 
 14  property_usage_ar        

In [209]:
missing_coords_df = exceptions[exceptions['Latitude'].isna() | exceptions['Longitude'].isna()][['area_name_en', 'area_name_ar_x']].drop_duplicates().reset_index(drop=True)


In [210]:
# Mapping for ordinal words to digits
ordinal_map = {
    'first': '1',
    'second': '2',
    'third': '3',
    'fourth': '4',
    'fifth': '5',
    'sixth': '6',
    'seventh': '7',
    'eighth': '8',
    'ninth': '9',
    'tenth': '10'
}

# Function to replace ordinal words with numbers
def replace_ordinals(name):
    for word, digit in ordinal_map.items():
        name = name.replace(word, digit).replace(word.capitalize(), digit)
    return name

# Apply the function
missing_coords_df['area_name_en_updated'] = missing_coords_df['area_name_en'].apply(replace_ordinals)


In [189]:
# One time run to find geo location via name. 

area_name = 'Al Dhagaya'

# Geocode the location
location = geocode(f"{area_name}, Dubai, UAE")

# Show result
if location:
    print(f"Address: {location.address}")
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Location not found.")
    
    

Address: الضغاية, ديرة, دبي, الإمارات العربية المتحدة
Latitude: 25.2726408, Longitude: 55.3002405


In [171]:
# area_name = 'Al Sufouh 2'

# # Geocode the location
# location = geocode(f"{area_name}, Dubai, UAE")

# # Show result
# if location:
#     print(f"Address: {location.address}")
#     print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
# else:
#     print("Location not found.")

In [211]:
def try_geocode(arabic_name, english_name):
    # Try with Arabic name
    location = geocode(f"{arabic_name}, Dubai, UAE")
    if location:
        return location
    # Fallback to English name
    return geocode(f"{english_name}, Dubai, UAE")

# Apply to the DataFrame
missing_coords_df["Location"] = missing_coords_df.apply(
    lambda row: try_geocode(row["area_name_ar_x"], row["area_name_en_updated"]), axis=1
)


In [213]:
missing_coords_df["Latitude"] = missing_coords_df["Location"].apply(lambda loc: loc.latitude if loc else None)
missing_coords_df["Longitude"] = missing_coords_df["Location"].apply(lambda loc: loc.longitude if loc else None)

In [214]:
exceptions_2 = missing_coords_df[missing_coords_df['Latitude'].isnull() | missing_coords_df['Longitude'].isnull()]

In [215]:
len(exceptions_2)

37

In [217]:
# These need to be manually updated. 

In [216]:
exceptions_2.head(37)

,area_name_en,area_name_ar_x,area_name_en_updated,Location,Latitude,Longitude
9,Al Goze First,القوز الاولى,Al Goze 1,None,NaN,NaN
12,Al Rega,الرقه,Al Rega,None,NaN,NaN
13,Muhaisanah Second,محيصنه الثانيه,Muhaisanah 2,None,NaN,NaN
17,Al Goze Fourth,القوز الرابعه,Al Goze 4,None,NaN,NaN
18,Al Goze Industrial Third,القوز الصناعيه الثالثه,Al Goze Industrial 3,None,NaN,NaN
19,Al Goze Industrial First,القوز الصناعيه الاولى,Al Goze Industrial 1,None,NaN,NaN
30,Al-Zarouniyyah,الزرعونيه,Al-Zarouniyyah,None,NaN,NaN
36,Al Rowaiyah First,الروية الأولى,Al Rowaiyah 1,None,NaN,NaN
43,Tawi Al Muraqqab,طوى مرقب,Tawi Al Muraqqab,None,NaN,NaN
45,Al-Murar Qadeem,المرر القديم,Al-Murar Qadeem,None,NaN,NaN


In [219]:
# Read manually updated file again 
df1 = pd.read_excel('C:/Users/DSHA-EXT/Desktop/Dubai Land Data/Python_Output_3_area_Lat_Long.xlsx')

In [230]:
exceptions_missing_coords_merge = pd.merge(exceptions,df1,how='left',on='area_name_en')
len(exceptions_missing_coords_merge)

21602

In [231]:
exceptions_missing_coords_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21602 entries, 0 to 21601
Data columns (total 61 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           21602 non-null  object 
 1   procedure_id             21602 non-null  int64  
 2   trans_group_id           21602 non-null  int64  
 3   trans_group_ar           21602 non-null  object 
 4   trans_group_en           21602 non-null  object 
 5   procedure_name_ar        21602 non-null  object 
 6   procedure_name_en        21602 non-null  object 
 7   instance_date            21602 non-null  object 
 8   property_type_id         21602 non-null  int64  
 9   property_type_ar         21602 non-null  object 
 10  property_type_en         21602 non-null  object 
 11  property_sub_type_id     4172 non-null   float64
 12  property_sub_type_ar     4172 non-null   object 
 13  property_sub_type_en     4172 non-null   object 
 14  property_usage_ar     

In [232]:
exceptions_missing_coords_merge = exceptions_missing_coords_merge.drop(columns=['Location_x','Latitude_x','Longitude_x','area_name_ar_x_y','Unnamed: 0',''], errors='ignore')


In [234]:
exceptions_missing_coords_merge[exceptions_missing_coords_merge['Latitude_y'].isnull() | exceptions_missing_coords_merge['Longitude_y'].isnull()][['area_name_en']].nunique()

area_name_en    18
dtype: int64

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427477 entries, 0 to 1427476
Data columns (total 55 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   transaction_id           1427477 non-null  object 
 1   procedure_id             1427477 non-null  int64  
 2   trans_group_id           1427477 non-null  int64  
 3   trans_group_ar           1427477 non-null  object 
 4   trans_group_en           1427477 non-null  object 
 5   procedure_name_ar        1427477 non-null  object 
 6   procedure_name_en        1427477 non-null  object 
 7   instance_date            1427477 non-null  object 
 8   property_type_id         1427477 non-null  int64  
 9   property_type_ar         1427477 non-null  object 
 10  property_type_en         1427477 non-null  object 
 11  property_sub_type_id     1124948 non-null  float64
 12  property_sub_type_ar     1124948 non-null  object 
 13  property_sub_type_en     1124948 non-null 

In [237]:
# First, ensure latitude/longitude are floats
exceptions_missing_coords_merge['Latitude_y'] = pd.to_numeric(exceptions_missing_coords_merge['Latitude_y'], errors='coerce')
exceptions_missing_coords_merge['Longitude_y'] = pd.to_numeric(exceptions_missing_coords_merge['Longitude_y'], errors='coerce')

# Create a lookup DataFrame with unique area_name_en keys
lookup_df = exceptions_missing_coords_merge[['area_name_en', 'Latitude_y', 'Longitude_y']].drop_duplicates(subset='area_name_en')

# Set area_name_en as index for fast lookup
lookup_df = lookup_df.set_index('area_name_en')

# Update main df based on area_name_en using .map
df['Latitude'] = df['area_name_en'].map(lookup_df['Latitude_y']).combine_first(df['Latitude'])
df['Longitude'] = df['area_name_en'].map(lookup_df['Longitude_y']).combine_first(df['Longitude'])


In [238]:
df[df['Latitude'].isnull() | df['Longitude'].isnull()][['area_name_en']].nunique()

area_name_en    19
dtype: int64

In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427477 entries, 0 to 1427476
Data columns (total 55 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   transaction_id           1427477 non-null  object 
 1   procedure_id             1427477 non-null  int64  
 2   trans_group_id           1427477 non-null  int64  
 3   trans_group_ar           1427477 non-null  object 
 4   trans_group_en           1427477 non-null  object 
 5   procedure_name_ar        1427477 non-null  object 
 6   procedure_name_en        1427477 non-null  object 
 7   instance_date            1427477 non-null  object 
 8   property_type_id         1427477 non-null  int64  
 9   property_type_ar         1427477 non-null  object 
 10  property_type_en         1427477 non-null  object 
 11  property_sub_type_id     1124948 non-null  float64
 12  property_sub_type_ar     1124948 non-null  object 
 13  property_sub_type_en     1124948 non-null 

In [242]:
df = df.drop(columns=['Area Name 2','area_name_ar_y','area_name_ar 2','Total Transactions','Location','Median Meter Sale Price'], errors='ignore')


In [243]:
df.to_csv('C:/Users/DSHA-EXT/Desktop/Dubai Land Data/Python_Output_4_final_lat_long.csv')

# Plot a Map

In [ ]:
df_filter = df[['']]

In [153]:
import folium

# Initialize map centered at Dubai's coordinates (approx.)
dubai_map = folium.Map(location=[25.276987, 55.296249], zoom_start=8)

# Define a function to determine color based on the area
def get_color(area_name):
    # You can customize this to add more areas or categories
    area_colors = {
        'Area 1': 'blue',
        'Area 2': 'green',
        'Area 3': 'red',
        'Area 4': 'purple',
    }
    # Default color if area is not in the map
    return area_colors.get(area_name, 'gray')

# Loop through the dataframe and add markers to the map
for index, row in exception_list.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=2,
            color=get_color(row['Area Name 2']),
            fill=True,
            fill_color=get_color(row['Area Name 2']),
            fill_opacity=0.2,
            popup=row['Area Name 2']  # Display area name on click
        ).add_to(dubai_map)

# Save the map to an HTML file
dubai_map.save("dubai_map_with_legend.html")

# If you want to display the map inline (in Jupyter notebooks)
dubai_map


In [128]:
transactions[['area_name_en']].nunique()

area_name_en    256
dtype: int64

In [149]:
areas_to_plot = exception_list[exception_list['Latitude'].notna()]

In [150]:
areas_to_plot.head()

,Unnamed: 0,Area Name,Area Name 2,area_name_ar,area_name_ar 2,Median Meter Sale Price,Total Transactions,Location,Latitude,Longitude
0,0,Jumeirah Second,Jumeirah 2,جميرا الثانيه,جميرا الثانيه,48415.890,206,Custom,25.200831,55.245202
1,1,Island 2,Island 2,جزيرة 2,جزيرة 2,45355.210,367,"جزيرة جميرا 2, دبي, الإمارات العربية المتحدة",25.197624,55.225824
2,2,Al Safouh Second,2 Al Sufouh,الصفوح الثانيه,الصفوح الثانيه,29831.260,1104,Custom,25.109536,55.172214
3,3,Zaabeel First,Za'abeel 1,زعبيل الاولى,زعبيل الاولى,29359.580,1993,Custom,25.218657,55.308387
4,4,Palm Jumeirah,Palm Jumeirah,نخلة جميرا,نخلة جميرا,26532.485,8882,"نخلة جميرا, دبي, الإمارات العربية المتحدة",25.118088,55.134201


In [154]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans

# Apply K-Means clustering (you can adjust the number of clusters as needed)
kmeans = KMeans(n_clusters=3, random_state=0)  # 3 clusters, you can adjust this
areas_to_plot['Cluster'] = kmeans.fit_predict(areas_to_plot[['Latitude', 'Longitude', 'Median Meter Sale Price']])

# Initialize map centered at Dubai's coordinates (approx.)
dubai_map = folium.Map(location=[25.276987, 55.296249], zoom_start=12)

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(dubai_map)

# Define color map based on clusters
cluster_colors = ['blue', 'green', 'red']  # Assign a color for each cluster

# Add markers with different colors based on cluster
for index, row in areas_to_plot.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        # Get color based on cluster from the 'Cluster' column
        cluster = row['Cluster']
        color = cluster_colors[cluster] if pd.notnull(cluster) else 'gray'  # Default to gray if no cluster

        # Add a marker to the cluster
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=2,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.2,
            popup=f"{row['Area Name 2']}<br>Price: {row['Median Meter Sale Price']} AED<br>Cluster: {cluster}",
        ).add_to(marker_cluster)

# Save the map to an HTML file
dubai_map.save("dubai_map_with_kmeans_clustering.html")

# If you want to display the map inline (in Jupyter notebooks)
dubai_map


C:\Users\DSHA-EXT\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\DSHA-EXT\AppData\Local\Temp\ipykernel_42840\595934694.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  areas_to_plot['Cluster'] = kmeans.fit_predict(areas_to_plot[['Latitude', 'Longitude', 'Median Meter Sale Price']])


In [140]:
areas_to_plot.head(100)

,Unnamed: 0,Area Name,Area Name 2,area_name_ar,area_name_ar 2,Median Meter Sale Price,Total Transactions,Location,Latitude,Longitude,Cluster
0,0,Jumeirah Second,Jumeirah 2,جميرا الثانيه,جميرا الثانيه,4.490996,206,Custom,0.433622,-0.526005,1.0
1,1,Island 2,Island 2,جزيرة 2,جزيرة 2,4.124306,367,"جزيرة جميرا 2, دبي, الإمارات العربية المتحدة",0.406634,-0.702055,1.0
2,2,Al Safouh Second,2 Al Sufouh,الصفوح الثانيه,الصفوح الثانيه,2.264431,1104,Custom,-0.334685,-1.189106,1.0
3,3,Zaabeel First,Za'abeel 1,زعبيل الاولى,زعبيل الاولى,2.207920,1993,Custom,0.583641,0.048037,1.0
4,4,Palm Jumeirah,Palm Jumeirah,نخلة جميرا,نخلة جميرا,1.869215,8882,"نخلة جميرا, دبي, الإمارات العربية المتحدة",-0.262717,-1.534454,1.0
...,...,...,...,...,...,...,...,...,...,...,...
117,117,Oud Al Muteena First,Oud Al Muteena 1,عود المطينة الأولى,عود المطينة الأولى,-0.938121,11,NaN,0.964439,1.314606,0.0
118,118,Al Mizhar Third,Al Mizhar 3,المزهر الثالثة,المزهر الثالثة,-0.941435,9,NaN,0.961271,1.469234,0.0
119,119,Al Rashidiya,Al Rashidiya,الراشديه,الراشديه,-0.942023,233,"الراشدية, دبي, الإمارات العربية المتحدة",0.611985,0.798636,0.0
120,120,Al Warqa Fourth,Al Warqa 4,الورقاء الرابعه,الورقاء الرابعه,-0.942098,14,NaN,0.344963,1.146070,0.0


In [165]:
# manual Updates 

# Area name with apostrophe
area_name = 'Umm Suqeim 1'

# Geocode the location
location = geocode(f"{area_name}, Dubai, UAE")

# Show result
if location:
    print(f"Address: {location.address}")
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Location not found.")

Address: أم سقيم 1, أم سقيم, دبي, الإمارات العربية المتحدة
Latitude: 25.1647586, Longitude: 55.2187285
